# Clean up North Carolina Ownership Data

This notebook combines the information received from the North Carolina Department of Health with the QCOR dataset.

In [1]:
import pandas as pd

## Load North Carolina State Data

In [2]:
RAW_OWNERSHIP_DIR = "../../data/ownership/raw/"
INTERMEDIATE_OWNERSHIP_DIR = "../../data/ownership/intermediate/"

In [3]:
nc_data = pd.read_excel(RAW_OWNERSHIP_DIR + "2021-10-15-nc-icfmr.xlsx", sheet_name="MHL_ExcelWebReport", skiprows=6)

In [4]:
nc_data.head().T

,0,1,2,3,4
RowNo.,1,2,3,4,5
FID #,922765,922412,954623,952105,954539
License #,MHL-001-008,MHL-001-007,MHL-001-031,MHL-001-027,MHL-001-030
Name of Licensee Legal Name,"Ralph Scott Lifeservices, Inc.","Ralph Scott Lifeservices, Inc.","Ralph Scott Lifeservices, Inc.","Ralph Scott Lifeservices, Inc.","Ralph Scott Lifeservices, Inc."
Unnamed: 4,NaN,NaN,NaN,NaN,NaN
DBA Name,"Ralph Scott Lifeservices, Inc.",Poplar Street Group Home,"Ralph Scott Lifeservices, Inc./Rosemont Street","Ralph Scott Lifeservices, inc.-Laramie Drive G...","Ralph Scott Lifeservices, Inc./Veterans Drive"
Unnamed: 6,NaN,NaN,NaN,NaN,NaN
Facility Contact Name,Brittany Mayak,Brittany Mayak,Brittany Mayak,Brittany Mayak,Brittany Mayak
Unnamed: 8,NaN,NaN,NaN,NaN,NaN
Facility Contact Number,(336) 227-1011,(336) 227-1011,(336) 227-1011,(336) 227-1011,(336) 227-1011


In [5]:
len(nc_data)

337

**We need to fix the CCN Numbers for a few facilities, we'll do that below**

In [6]:
nc_data["CCN Number"].isnull().sum()

58

In [7]:
ccn_fixes = pd.read_csv(INTERMEDIATE_OWNERSHIP_DIR + "unmatched-nc-federal-icfs-resolution.csv")

In [8]:
ccn_fixes.head()

,name,provider_id,DBA name
0,CASWELL CENTER,34G001,NaN
1,MURDOCH DEVELOPMENTAL CENTER,34G002,NaN
2,J. IVERSON RIDDLE DEVELOPMENTAL CENTER,34G003,NaN
3,O'BERRY NEURO-MEDICAL TREATMENT CENTER,34G004,NaN
4,BEAR CREEK,34G006,Bear Creek


In [9]:
def clean_ccn(row):
    if pd.isnull(row["CCN Number"]):
        if row["DBA Name"] in ccn_fixes["DBA name"].values:
            return ccn_fixes.loc[lambda x: x["DBA name"] == row["DBA Name"]]["provider_id"].values[0]
        else:
            print(row['DBA Name'])
            return None
    else:
        return row["CCN Number"]

*Rayside B and Rayside A are one facility in the QCOR data*

In [10]:
nc_data["clean_ccn"] = nc_data.apply(lambda x: clean_ccn(x), axis=1)

Rayside B


## Load QCOR Data and Join

In [11]:
facs = pd.read_csv("../../data/qcor/facilities.csv")
nc_facs = facs[lambda x: x["state"] == "NC"].copy()

In [12]:
nc_facs.head()

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date
3903,CASWELL CENTER,34G001,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"2415 W. VERNON AVENUE\nKINSTON, NC 28501",252 559-5100,1974-03-31,405,No,Government,NaN,NaN
3904,MURDOCH DEVELOPMENTAL CENTER,34G002,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"1600 EAST C STREET\nBUTNER, NC 27509",919 575-7734,1974-03-31,657,No,Government,NaN,NaN
3905,J. IVERSON RIDDLE DEVELOPMENTAL CENTER,34G003,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"300 ENOLA ROAD\nMORGANTON, NC 28655",828 433-2711,1974-03-31,490,No,Government,NaN,NaN
3906,O'BERRY NEURO-MEDICAL TREATMENT CENTER,34G004,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"400 OLD SMITHFIELD RD\nGOLDSBORO, NC 27530",919 731-3661,1974-03-31,485,Yes,Government,NaN,NaN
3907,BEAR CREEK,34G006,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"5840 GREENWOOD AVENUE\nLA GRANGE, NC 28551",252 566-9181,1975-06-01,113,No,For Profit,NaN,NaN


In [13]:
len(nc_facs)

338

In [14]:
len(nc_data)

337

In [15]:
nc_ownership = (
    pd
    .merge(
        nc_facs,
        nc_data[["clean_ccn", "Name of Licensee Legal Name", "Owner"]],
        left_on="provider_id",
        right_on="clean_ccn",
        how="left"
    )
    .drop_duplicates(subset=["provider_id"]) # NC treats some facilities as two in its state data, so we drop duplicates
    .rename(columns={"Name of Licensee Legal Name": "legal_owner"})
    .assign(
        is_brightspring = lambda df: df["legal_owner"].isin([
            "VOCA Corporation of North Carolina",
            "EduCare Community Living Corporation-North Carolina",
            "ResCare of North Carolina-EduCare Community Living"
        ])
    )
)

There are five facilities that don't merge. North Carolina said via email on Nov. 17, 2021 that Tammy Lynn is owned by TLC Operations, Inc. and that the others are owned by the state. Thus the below fields showing that none are owned by BrightSpring are correct.

In [16]:
nc_ownership[
    lambda x: x["legal_owner"].isnull()
]

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,clean_ccn,legal_owner,Owner,is_brightspring
0,CASWELL CENTER,34G001,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"2415 W. VERNON AVENUE\nKINSTON, NC 28501",252 559-5100,1974-03-31,405,No,Government,NaN,NaN,NaN,NaN,NaN,False
1,MURDOCH DEVELOPMENTAL CENTER,34G002,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"1600 EAST C STREET\nBUTNER, NC 27509",919 575-7734,1974-03-31,657,No,Government,NaN,NaN,NaN,NaN,NaN,False
2,J. IVERSON RIDDLE DEVELOPMENTAL CENTER,34G003,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"300 ENOLA ROAD\nMORGANTON, NC 28655",828 433-2711,1974-03-31,490,No,Government,NaN,NaN,NaN,NaN,NaN,False
3,O'BERRY NEURO-MEDICAL TREATMENT CENTER,34G004,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"400 OLD SMITHFIELD RD\nGOLDSBORO, NC 27530",919 731-3661,1974-03-31,485,Yes,Government,NaN,NaN,NaN,NaN,NaN,False
112,TAMMY LYNN CENTER/CHILDREN,34G124,Intermediate Care Facilities for Individuals w...,(IV) Atlanta,NC,"743 & 745 CHAPPELL DRIVE\nRALEIGH, NC 27606",919 832-3909,1989-11-15,20,No,Non-Profit,NaN,NaN,NaN,NaN,NaN,False


In [17]:
len(nc_ownership)

338

In [18]:
nc_ownership[[
    "name", "provider_id", "type", "region", "state", "address",
    "phone", "address", "particip_date", "certified_beds",
    "hospital_based", "ownership_type", "termination_code",
    "termination_date", "legal_owner", "is_brightspring"
]].to_csv("../../data/ownership/final/nc.csv", index=False)

---

---

---